<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting-up-imports" data-toc-modified-id="Setting-up-imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting up imports</a></span></li><li><span><a href="#Setting-up-Constant-Hyperparameters" data-toc-modified-id="Setting-up-Constant-Hyperparameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up Constant Hyperparameters</a></span></li><li><span><a href="#Setting-up-Parameters-and-Functions-for-Training" data-toc-modified-id="Setting-up-Parameters-and-Functions-for-Training-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up Parameters and Functions for Training</a></span><ul class="toc-item"><li><span><a href="#Hyperparameters-Search-Space" data-toc-modified-id="Hyperparameters-Search-Space-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Hyperparameters Search Space</a></span></li><li><span><a href="#Creating-the-training-function" data-toc-modified-id="Creating-the-training-function-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creating the training function</a></span></li><li><span><a href="#Creating-the-evaluation-function" data-toc-modified-id="Creating-the-evaluation-function-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Creating the evaluation function</a></span></li></ul></li><li><span><a href="#Running-the-training" data-toc-modified-id="Running-the-training-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running the training</a></span><ul class="toc-item"><li><span><a href="#Loading-data-for-training" data-toc-modified-id="Loading-data-for-training-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Loading data for training</a></span></li><li><span><a href="#Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm" data-toc-modified-id="Configuring-the-Tuner-with-a-Scheduler-and-a-Search-Algorithm-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Configuring the Tuner with a Scheduler and a Search Algorithm</a></span></li><li><span><a href="#Running-the-Tuner" data-toc-modified-id="Running-the-Tuner-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Running the Tuner</a></span></li></ul></li><li><span><a href="#Evaluating-the-best-Results" data-toc-modified-id="Evaluating-the-best-Results-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Evaluating the best Results</a></span></li></ul></div>

# Setting up imports

In [1]:
import os

import torch
from torch.nn import CrossEntropyLoss
from torch.nn.functional import normalize
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision.transforms import GaussianBlur
# from torchvision.transforms.functional import invert
from torchvision.models import vision_transformer

import ray
from ray import tune
from ray.air import session, RunConfig
from ray.air.checkpoint import Checkpoint
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch


from dataset import POCDataReader, data_augment_, POCDataset
from metrics import Metrics, EvaluationMetrics
from models import UNet
from loss import *
from pipelines import InputPipeline, SumFilters
from pipelines.filters import *
from train import training_loop, validation_loop
from train_tqdm import evaluation_loop


# Setting up Constant Hyperparameters

In [2]:
EPOCHS = 15
NUM_SAMPLES = 150
NUM_MODEL_TEST = 10

NUM_AUGMENT = 1

LOAD_DATA_ON_GPU = True
GPUS_PER_TRIAL = 1
CPUS_PER_TRIAL = 20

##### Selecting Cuda device

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


# Setting up Parameters and Functions for Training

## Hyperparameters Search Space

In [4]:
search_space = {
    "Network": UNet,
    "Optimizer": Adam,
    
    "Learning Rate": 1e-4,     #tune.qloguniform(1e-5, 1e-2, 5e-6),
    "Batch Size": 4,           #tune.qrandint(2, 8, 2),

    "Pixel Loss": tune.choice([CrossEntropyLoss(weight=torch.tensor([.3, .7])), FocalLoss(weight=torch.tensor([.3, .7]), gamma=2)]),
    "Volume Loss": tune.choice([JaccardLoss(), TverskyLoss(alpha=0.3, beta=0.7), FocalTverskyLoss(alpha=0.3, beta=0.7, gamma=2)]),
    "Combine Loss": tune.choice([CombinedLoss, BorderedLoss, PixelLoss, VolumeLoss]),
    
    "Negative Mining": tune.choice([True, False]),
    "Smooth Labeling": tune.choice([True, False]),

    "Input Filter": normalize,  #tune.choice([None, invert]),
    "Input Layer": FrangiFilter(), #tune.choice([FrangiFilter(), SatoFilter(), SumFilters(FrangiFilter(), SatoFilter())]), #DINOFilter()
}

## Creating the training function

In [5]:
def train(config, train_data, val_data):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    inpip = InputPipeline(
        transformer=config["Input Filter"],
        layer_transformer=config["Input Layer"])
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    train_dataset = POCDataset(
        train_data,
        transform=inpip,
        target_transform= GaussianBlur(kernel_size=3, sigma=0.7) if config["Smooth Labeling"] else None,
        negative_mining=config["Negative Mining"],
        load_on_gpu=LOAD_DATA_ON_GPU)
    train_dataset.precompute_transform()

    if LOAD_DATA_ON_GPU:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
        )
    else:
        training_dataloader = DataLoader(
            train_dataset,
            batch_size=int(config["Batch Size"]),
            sampler=train_dataset.sampler,
            shuffle= True if train_dataset.sampler is None else None,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    val_dataset = POCDataset(
        val_data, 
        transform=inpip, 
        target_transform=None, 
        negative_mining=False,
        load_on_gpu=LOAD_DATA_ON_GPU)
    val_dataset.precompute_transform()
    
    if LOAD_DATA_ON_GPU:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True)
    else:
        validation_dataloader = DataLoader(
            val_dataset,
            batch_size=int(config["Batch Size"]),
            shuffle=True,
            num_workers=CPUS_PER_TRIAL//2,
            pin_memory=True,
            pin_memory_device=device)

    model = config["Network"](n_channels=inpip.nb_channel, n_classes=2, bilinear=True, crop=False)
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    loss_fn = config["Combine Loss"](config["Pixel Loss"], config["Volume Loss"]).to(device)
    optimizer = config["Optimizer"](model.parameters(), lr=config["Learning Rate"], betas=(0.9, 0.99))
    lr_scheduler = CosineAnnealingLR(optimizer, T_max=EPOCHS)

    loaded_checkpoint = session.get_checkpoint()
    if loaded_checkpoint:
        with loaded_checkpoint.as_directory() as loaded_checkpoint_dir:
            model_state, optimizer_state, scheduler_state = torch.load(os.path.join(loaded_checkpoint_dir, "checkpoint.pt"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        lr_scheduler.load_state_dict(scheduler_state)

    train_metrics = Metrics(
        buffer_size=len(training_dataloader),
        mode="Training",
        hyperparam=config,
        device=device)

    val_metrics = Metrics(
        buffer_size=len(validation_dataloader),
        mode="Validation",
        hyperparam=config,
        device=device)


    for epoch in range(1, EPOCHS+1):  # loop over the dataset multiple times
        training_loop(epoch, training_dataloader, model, loss_fn, optimizer, lr_scheduler, train_metrics, device)
        validation_loop(epoch, validation_dataloader, model, loss_fn, val_metrics, device)

        # Here we save a checkpoint. It is automatically registered with
        # Ray Tune and can be accessed through `session.get_checkpoint()`
        # API in future iterations.
        os.makedirs("model", exist_ok=True)
        torch.save((model.state_dict(), optimizer.state_dict(), lr_scheduler.state_dict()), "model/checkpoint.pt")
        checkpoint = Checkpoint.from_directory("model")
        session.report(metrics=val_metrics.get_metrics(epoch), checkpoint=checkpoint)

    train_metrics.close_tensorboard()
    val_metrics.close_tensorboard()


## Creating the evaluation function

In [6]:
def evaluate(test_data, result):

    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    inpip = InputPipeline(
        transformer=result.config["Input Filter"], 
        layer_transformer=result.config["Input Layer"])
    if LOAD_DATA_ON_GPU:
        inpip = inpip.to(device)

    test_dataset = POCDataset(test_data, transform=inpip, target_transform=None, negative_mining=False, load_on_gpu=LOAD_DATA_ON_GPU)
    
    if LOAD_DATA_ON_GPU:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)
    else:
        evaluation_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=20, pin_memory=True, pin_memory_device=device)

    best_trained_model = result.config["Network"](n_channels=inpip.nb_channel, n_classes=2, bilinear=True, crop=False).to(device)

    checkpoint_path = os.path.join(result.checkpoint.to_directory(), "checkpoint.pt")
    model_state, _, _ = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    test_metrics = EvaluationMetrics(
        buffer_size=len(evaluation_dataloader),
        hyperparam=result.config,
        epochs=result.metrics["Epoch"],
        device=device)

    evaluation_loop(dataloader=evaluation_dataloader, model=best_trained_model, metric=test_metrics, device=device)


# Running the training

## Loading data for training

In [7]:
data_reader = POCDataReader(root_dir="../data/POC", load_on_gpu=False, verbose=True)
train_data, val_data, test_data = data_reader.split([0.7, 0.1, 0.2])
data_augment_(train_data, n=NUM_AUGMENT, load_on_gpu=False, verbose=True)

Loading dataset into RAM:   0%|          | 0/2744 [00:00<?, ?it/s]

	- Loading done, RAM used: 7.20GiB / free: 28.72GiB / total: 125.40GiB
	- Got a total of 2744 images.


Expending the dataset 1 more times:   0%|          | 0/1920 [00:00<?, ?it/s]

	- Augmentation done, RAM used: 9.94GiB / free: 25.98GiB / total: 125.40GiB
	- Got 1920 new images and a total of 3840 images.


## Configuring the Tuner with a Scheduler and a Search Algorithm

In [8]:
scheduler = ASHAScheduler(max_t=EPOCHS, grace_period=2, reduction_factor=2)
search_algo = HyperOptSearch()

tune_config = tune.TuneConfig(
    metric="CrackIoU",
    mode="max",
    num_samples=NUM_SAMPLES,
    scheduler=scheduler,
    search_alg=search_algo)

tuner = tune.Tuner(
    tune.with_resources(
        tune.with_parameters(train, train_data=train_data, val_data=val_data),
        resources={"cpu": CPUS_PER_TRIAL, "gpu": GPUS_PER_TRIAL}),
    tune_config=tune_config,
    param_space=search_space,
    run_config=RunConfig(local_dir="~/POC-Project/ray_results"))

## Running the Tuner

In [ ]:
results = tuner.fit()

2023-03-30 17:19:05,676	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Trial name,CrackIoU,Epoch,Loss,MeanIoU,Tversky,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_004b8015,0.8149253726005554,8,0.04847266152501106,0.9061765074729919,0.8969734311103821,2023-03-31_06-50-41,True,,5819102d969d4c45adca592705b26803,pirl-PowerEdge-T640,8,141.223.108.122,7282,True,3940.255780696869,378.9906711578369,3940.255780696869,1680213041,0,,8,004b8015,0.029394149780273438
train_0118226b,0.8309852480888367,15,0.08563954383134842,0.914360523223877,0.9103679060935974,2023-03-30_22-32-44,True,,5819102d969d4c45adca592705b26803,pirl-PowerEdge-T640,15,141.223.108.122,7282,True,6563.101820707321,376.9616069793701,6563.101820707321,1680183164,0,,15,0118226b,0.029394149780273438
train_054146f9,0.1494825780391693,2,0.0013666617451235652,0.5473992824554443,0.3472917675971985,2023-03-31_01-30-42,True,,5819102d969d4c45adca592705b26803,pirl-PowerEdge-T640,2,141.223.108.122,7282,True,1664.1846039295197,376.4370427131653,1664.1846039295197,1680193842,0,,2,054146f9,0.029394149780273438
train_07e61d56,,,,,,2023-03-30_20-03-10,,,2ffe0ec47f5f4fde8e8d9c9d2743e6cd,pirl-PowerEdge-T640,,141.223.108.122,9421,,,,,1680174190,,,,07e61d56,
train_0e80ab08,0.6886513233184814,2,0.014957666397094727,0.8417055606842041,0.8485428094863892,2023-03-31_09-53-24,True,,e6779c8fb99c48058a301282b98ff3ad,pirl-PowerEdge-T640,2,141.223.108.122,10285,True,1669.5488002300262,377.77550315856934,1669.5488002300262,1680224004,0,,2,0e80ab08,0.02836132049560547
train_0fd7a8b9,0.8357040286064148,15,0.04300940781831741,0.9167305827140808,0.9111551642417908,2023-03-31_01-02-57,True,,5819102d969d4c45adca592705b26803,pirl-PowerEdge-T640,15,141.223.108.122,7282,True,6598.506895065308,379.2771954536438,6598.506895065308,1680192177,0,,15,0fd7a8b9,0.029394149780273438
train_14b842ff,,,,,,2023-03-30_19-08-08,,,da0098dce76d44e49c1ac93ef1bb1ab9,pirl-PowerEdge-T640,,141.223.108.122,7195,,,,,1680170888,,,,14b842ff,
train_16db9183,0.7772772908210754,4,0.11303821951150894,0.886961817741394,0.8845995664596558,2023-03-30_23-28-08,True,,e6779c8fb99c48058a301282b98ff3ad,pirl-PowerEdge-T640,4,141.223.108.122,10285,True,2406.9950964450836,374.84212136268616,2406.9950964450836,1680186488,0,,4,16db9183,0.02836132049560547
train_187244da,0.7142854928970337,2,0.14507193863391876,0.8549280762672424,0.8289235830307007,2023-03-31_01-58-22,True,,5819102d969d4c45adca592705b26803,pirl-PowerEdge-T640,2,141.223.108.122,7282,True,1660.4649925231934,376.0921733379364,1660.4649925231934,1680195502,0,,2,187244da,0.029394149780273438
train_2a8294b2,0.7837362289428711,8,0.005980351474136114,0.8901745676994324,0.9041333794593811,2023-03-30_20-43-20,True,,5819102d969d4c45adca592705b26803,pirl-PowerEdge-T640,8,141.223.108.122,7282,True,3963.6314566135406,378.3460123538971,3963.6314566135406,1680176600,0,,8,2a8294b2,0.029394149780273438


2023-03-30 17:47:01,883	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.VolumeLoss'>, 'Input Filter': <function normalize at 0x7fe27486caf0>, 'Input Layer': FrangiFilter, 'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Pixel Loss': FocalLoss, 'Volume Loss': FocalTverskyLoss}
2023-03-30 19:08:08,456	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.CombinedLoss'>, 'Input Filter': <function normalize at 0x7fe27486caf0>, 'Input Layer': FrangiFilter, 'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Pixel Loss': FocalLoss, 'Volume Loss': TverskyLoss}
2023-03-30 19:08:17,036	ERROR trial_runner.py:1062 -- Trial train_14b842ff: Error processing event.
ray.exceptions.RayTaskError(RuntimeError): ray::ImplicitFunc.train() (pid=7195, ip=141.223.108.122,

2023-03-30 22:48:01,218	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.BorderedLoss'>, 'Input Filter': <function normalize at 0x7fe27486caf0>, 'Input Layer': FrangiFilter, 'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Pixel Loss': CrossEntropyLoss(), 'Volume Loss': FocalTverskyLoss}
2023-03-30 23:12:59,273	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.VolumeLoss'>, 'Input Filter': <function normalize at 0x7fe27486caf0>, 'Input Layer': FrangiFilter, 'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Pixel Loss': FocalLoss, 'Volume Loss': JaccardLoss}
2023-03-30 23:28:08,314	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.VolumeLoss'>, 'Input Filter': 

2023-03-31 05:49:10,466	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.VolumeLoss'>, 'Input Filter': <function normalize at 0x7fe27486caf0>, 'Input Layer': FrangiFilter, 'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Pixel Loss': FocalLoss, 'Volume Loss': JaccardLoss}
2023-03-31 06:16:47,730	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.VolumeLoss'>, 'Input Filter': <function normalize at 0x7fe27486caf0>, 'Input Layer': FrangiFilter, 'Network': <class 'models.unet.UNet'>, 'Optimizer': <class 'torch.optim.adam.Adam'>, 'Pixel Loss': CrossEntropyLoss(), 'Volume Loss': TverskyLoss}
2023-03-31 06:44:30,468	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'Combine Loss': <class 'loss.loss.CombinedLoss'>, 'Input Filter': <func

# Evaluating the best Results

In [ ]:
best_result = results.get_best_result(metric="CrackIoU", mode="max", scope="all")  # Get best result object
print("Best trial config: {}".format(best_result.config))
print("Best trial final validation loss: {}".format(best_result.metrics["Loss"]))
print("Best trial final validation CrackIoU: {}".format(best_result.metrics["CrackIoU"]))

for result in results:
    evaluate(test_data=test_data, result=result)